## Clone CSVs

In [1]:
!wget https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/PCG_Refined.csv
!wget https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/Queries.csv

--2024-12-15 21:29:37--  https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/PCG_Refined.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120894 (118K) [text/plain]
Saving to: ‘PCG_Refined.csv’

PCG_Refined.csv     100%[===================>] 118.06K  --.-KB/s    in 0.01s   

2024-12-15 21:29:38 (7.99 MB/s) - ‘PCG_Refined.csv’ saved [120894/120894]

--2024-12-15 21:29:38--  https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/Queries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

## Dependencies

In [2]:
!pip install langchain-chroma==0.1.4
!pip install langchain_core==0.3.24
!pip install langchain_huggingface==0.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.9 MB/s eta 0:0

## Embedding model :

- You can choose  any embedding model from **Huggingface** or **Ollama** , in our case we prefer using **Huggingface** because of the huge amount of availible models.
- Discover Embedding Modles <a href = "https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=likes">here</a> or see <a href = "https://huggingface.co/spaces/mteb/leaderboard">Embedding models LeaderBoard</a>

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

Embedding_name = "dangvantuan/french-document-embedding"
model_kwargs = {'device': 'cuda','trust_remote_code': True }
encode_kwargs = {'normalize_embeddings': False}
                #'precision': "float32",}
embed_model = HuggingFaceEmbeddings(model_name=Embedding_name,model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at infgrad/jasper_en_vision_language_v1 were not used when initializing JasperVL: ['vision_model.vision_model.embeddings.patch_embedding.bias', 'vision_model.

## CHROMA_Refined

### Build

In [ ]:
from langchain_chroma import Chroma
from uuid import uuid4
from langchain_core.documents import Document

In [ ]:
def CreateDocuments(df):
  documents = []
  for index, row in df.iterrows():
    document = Document(page_content = row["Definition"],
                        metadata={"AccountID": row["Id"],"AccountTitle":row["Title"]})
    documents.append(document)
  return documents

In [ ]:
import pandas as pd

Accounts_Refined = pd.read_csv("PCG_Refined.csv")
Accounts_Refined.head()

In [ ]:
collection = "PCG_Accounts_Refined"
folder = "./CHROMA_Refined"+"_"+Embedding_name.split("/")[-1]

In [ ]:
# Initialize

vector_store = Chroma(
    collection_name=collection,
    embedding_function=embed_model,
    persist_directory=folder,
    collection_metadata={"hnsw:space": "cosine"},# To insure that Similarities will be Normilized between 0 and 1
)

# Add Documents
Documents = CreateDocuments(Accounts_Refined)
uuids = [str(uuid4()) for _ in range(len(Documents))]
vector_store.add_documents(documents=Documents, ids=uuids)

### Evaluate

In [ ]:
db_Refined = Chroma(persist_directory="CHROMA_Refined"+"_"+Embedding_name.split("/")[-1],
                embedding_function=embed_model,
                collection_name="PCG_Accounts_Refined",
                collection_metadata={"hnsw:space": "cosine"})

def Retrieve_Refined(Query):
  Outputs = {}
  results = db_Refined.similarity_search_with_relevance_scores(Query, k=5)
  for res, score in results:
    #print(res.metadata)
    Id = res.metadata["AccountID"]
    Outputs[Id] = score
  return Outputs

In [ ]:
import pandas as pd
Queries = pd.read_csv("Queries.csv")
Queries.head()

- Test with a Query

In [ ]:
query = Queries.loc[1]["Query"]
correctID = Queries.loc[1]["AccountID"]

print("Query : ", query)
print("Correct ID : ", correctID)

In [ ]:
Retrieve_Refined(query)

- Test on all Queries

In [ ]:
from tqdm import tqdm


def Calculate_value_Refined(Outputs,k):
    if k not in Outputs.keys():
        return 0
    sim_score_k = Outputs[k]
    return sim_score_k


def Evaluate_Refined(Queries):
    Scores = []
    for index, row in tqdm(Queries.iterrows(), total=Queries.shape[0], desc="Refined_Evaluating..."):
        query = row["Query"]
        correctID = row["AccountID"]
        results = Retrieve_Refined(query)
        score = Calculate_value_Refined(results, correctID)
        Scores.append(score)

    Queries["Similarity"] = Scores
    Queries.to_csv("Refined_Evaluation.csv",index = False)


In [ ]:
Evaluate_Refined(Queries)

- Where there are a 0 means that the retiever didn't return the Correct account ID
- Where there are a value(!=0) it corresponding to the Similarity with Correct Definition.

In [ ]:
Refined_Evaluation = pd.read_csv("Refined_Evaluation.csv")
Refined_Evaluation.head()

In [ ]:
Average = Refined_Evaluation["Similarity"].mean()
zero_count = (Refined_Evaluation['Similarity'] == 0).sum()

print("Average Quality is :",Average)
print("Number of cases with 0 :",zero_count)

## Save The bechmarking

In [ ]:
# Run the following line for one time to create the dataframe after that just read the last saved csv
#embedding_results = pd.DataFrame(columns=['Embedding_modle', 'Average_Quality', 'Numberof_Zeros'])
embedding_results = pd.read_csv('EmbeddingM_Bench_results.csv')

In [ ]:
embedding_results

In [ ]:
new_data = {
    'Embedding_modle': Embedding_name,
    'Average_Quality': Average,
    'Numberof_Zeros': zero_count
}
new_row = pd.DataFrame([new_data])
embedding_results = pd.concat([embedding_results, new_row], ignore_index=True)
embedding_results.to_csv('EmbeddingM_Bench_results.csv', index=False)

In [ ]:
embedding_results